In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
path = "E:/CS_Master_Degree_UIUC/CS410_Text_Information_system/Project/Project Submission/CourseProject/Dataset/"
titlesFP = pd.read_csv(path + "titlesFP2000.csv") # closed sequential patterns found by PySpark PrefixSpan algorithm
dblp_titles = pd.read_table(path + "DBLP2000_preprocessed_titles.txt", header = None) # preprocess titles from DBLP dataset

In [3]:
titlesFP['title_pattern'] = titlesFP['title_pattern'].astype('str')
dblp_titles.columns = ['title']
dblp_titles['title'] = dblp_titles['title'].astype('str')

In [4]:
titlesFP.sort_values('Freq', ascending = False)

,title_pattern,Freq
450,system,372
285,use,354
676,model,298
677,network,208
1054,algorithm,190
...,...,...
1452,video segment,4
1453,video sequenc,4
806,way,4
1457,activ system,4


In [5]:
dblp_titles

,title
0,solut queue variant
1,nonapproxim result partial observ markov decis...
2,regular lemma applic graph theori
3,schema base approxim algorithm
4,adapt gener lisp tutor
...,...
3999,probabilist algorithm effici grasp fixtur
4000,arachnoid tourist hindsight
4001,domain name exchang share registri system
4002,jatlit java agent infrastructur messag rout


In [6]:
def find_pattern(title_pattern, title):
    output = False
    if title.find(title_pattern) != -1:
        output = True
    if title.find(title_pattern) == -1: 
        tp = title_pattern.split(' ')
        if len(tp) > 1 and all(title.find(p) != -1 for p in tp):
            output = True
    return output

In [7]:
# Find index of a title pattern in the dblp titles
def pattern_index(p, dblp_titles):
    '''
    Given a closed sequential pattern p of titles and the preprocess dblp_titles dataset,
    Output the indice of the transactions that include the pattern, p.
    '''
    
    return dblp_titles.loc[dblp_titles['title'].apply(lambda t: find_pattern(p, t))].index.tolist()

In [8]:
# Find index of all title patterns in the title_spark dataset
start =  time.time()
titlesFP["transaction_index"] = titlesFP['title_pattern'].apply(lambda x: pattern_index(x, dblp_titles))
end = time.time()
print((end-start)/60)

0.17767868439356485


In [9]:
titlesFP

,title_pattern,Freq,transaction_index
0,home,5,"[1977, 2509, 2601, 2602, 3078, 3126, 3854]"
1,perceptu,4,"[1044, 1450, 1579, 3112]"
2,net,22,"[45, 60, 85, 109, 118, 172, 174, 175, 178, 180..."
3,grid,12,"[364, 365, 375, 517, 952, 953, 1359, 1363, 212..."
4,delay,11,"[727, 1246, 2339, 2624, 2778, 3025, 3608, 3671..."
...,...,...,...
1908,learn system,9,"[70, 87, 88, 1525, 1794, 1798, 1875, 2063, 206..."
1909,learn model,4,"[995, 2063, 2266, 2868, 3147, 3345]"
1910,learn algorithm,5,"[88, 1442, 1824, 2077, 2700, 3883]"
1911,learn function,4,"[1093, 3349, 3352, 3645]"


In [10]:
# Check if all title patterns have transaction index found
titlesFP.loc[titlesFP['transaction_index'].apply(lambda x: len(x)==0)]

,title_pattern,Freq,transaction_index
248,"""""",11,[]


In [11]:
# Remove the above title pattern
broken = titlesFP.loc[titlesFP['transaction_index'].apply(lambda x: len(x)==0)].index.values
titlesFP = titlesFP.drop(broken)

In [12]:
titlesFP

,title_pattern,Freq,transaction_index
0,home,5,"[1977, 2509, 2601, 2602, 3078, 3126, 3854]"
1,perceptu,4,"[1044, 1450, 1579, 3112]"
2,net,22,"[45, 60, 85, 109, 118, 172, 174, 175, 178, 180..."
3,grid,12,"[364, 365, 375, 517, 952, 953, 1359, 1363, 212..."
4,delay,11,"[727, 1246, 2339, 2624, 2778, 3025, 3608, 3671..."
...,...,...,...
1908,learn system,9,"[70, 87, 88, 1525, 1794, 1798, 1875, 2063, 206..."
1909,learn model,4,"[995, 2063, 2266, 2868, 3147, 3345]"
1910,learn algorithm,5,"[88, 1442, 1824, 2077, 2700, 3883]"
1911,learn function,4,"[1093, 3349, 3352, 3645]"


In [ ]:
output_path = path
titlesFP.to_csv(output_path + "titlesFP2000_with_index.csv", index = False)